## Part 1: Setup

**Step 1:** Import the relevant packages and set Seaborn/Matplotlib hyperparameters.

In [ ]:
import holidays
import os

import matplotlib.dates as md
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from scipy import stats
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from tqdm import notebook

plt.style.use("fivethirtyeight")
sns.set(style="whitegrid", palette="muted")

plt.rcParams["axes.labelsize"] = 26
plt.rcParams["axes.titlesize"] = 26
plt.rcParams["figure.figsize"] = 16, 10
plt.rcParams["xtick.labelsize"] = 22
plt.rcParams["ytick.labelsize"] = 22

np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.2f}'.format

%config Completer.use_jedi = False

**Step 2:** Define the location of our data as well as the relevant columns that we would like to include.

In [ ]:
house_number = 12

cols_REFIT = [
    "Time",
    "Aggregate",
    "Appliance1",
    "Appliance2",
    "Appliance3",
    "Appliance4",
    "Appliance5",
    "Appliance6",
    "Appliance7",
    "Appliance8",
    "Appliance9",
]

data_directory_REFIT = os.path.join("Data", "REFIT")
data_directory_Solcast = os.path.join("Data", "Solcast_REFIT")

house = f"CLEAN_House{house_number}.csv"
solcast_15 = "Solcast_REFIT_15.csv"

file_destination_REFIT = os.path.join(data_directory_REFIT, house)
file_destination_Solcast = os.path.join(data_directory_Solcast, solcast_15)

**Step 3:** Read in the data and save it to a dataframe.

In [ ]:
df_REFIT = pd.read_csv(file_destination_REFIT, index_col=0, parse_dates=True, usecols=cols_REFIT)
df_Solcast = pd.read_csv(file_destination_Solcast, index_col=0, parse_dates=True)

df_Solcast.index = df_Solcast.index.rename("Time")
df_Solcast.index = pd.to_datetime(df_Solcast.index).tz_localize(None)

cols_REFIT.remove("Time")

## Part 2: Minor data transformation(s)
**Step 1.1:** Scale the data in range between 0 and 1 (if necessary).

In [ ]:
# minmax_REFIT = MinMaxScaler()
# minmax_Solcast = MinMaxScaler()

# df_REFIT[cols_REFIT] = minmax_REFIT.fit_transform(df_REFIT[cols_REFIT])
# df_Solcast[cols_Solcast] = minmax_Solcast.fit_transform(df_Solcast[cols_Solcast])

**Step 1.2:** Standardize data by removing the mean and scaling to unit variance (if necessary).

In [ ]:
standardscale_REFIT = StandardScaler()
# standardscale_Solcast = StandardScaler()

df_REFIT[cols_REFIT] = standardscale_REFIT.fit_transform(df_REFIT[cols_REFIT])
# df_Solcast[cols_Solcast] = standardscale_Solcast.fit_transform(df_Solcast[cols_Solcast])

**Step 2:** Create a copy of our REFIT dataframe that is resampled into a resolution of 15 minutes and drop any days that contain an incomplete number of values.

In [ ]:
df_REFIT_resampled = df_REFIT.resample("15min").mean()
df_REFIT_resampled = df_REFIT_resampled.dropna()

mask = df_REFIT_resampled.groupby(df_REFIT_resampled.index.date).size()
mask = mask[mask < 96].index.to_list()

df_REFIT_resampled = df_REFIT_resampled[~df_REFIT_resampled.index.floor("D").isin(mask)]

**Step 3:** Create a third dataframe that is the result of merging the Solcast dataframe with the REFIT dataframe.

In [ ]:
df_Merged = pd.merge(left=df_Solcast, left_on=df_Solcast.index, right=df_REFIT_resampled, right_on=df_REFIT_resampled.index)

cols_Merged = [
    "PeriodStart",
    "Period",
    "Appliance1",
    "Appliance2",
    "Appliance3",
    "Appliance4",
    "Appliance5",
    "Appliance6",
    "Appliance7",
    "Appliance8",
    "Appliance9",
]

df_Merged.drop(cols_Merged, axis=1, inplace=True)
df_Merged.rename(columns={"key_0": "Time"}, inplace=True)
df_Merged = df_Merged.set_index("Time")
df_Merged.index = pd.to_datetime(df_Merged.index)
df_Merged.head()

**Step 4:** Append public holidays to our merged dataframe.

In [ ]:
UK_holidays = holidays.UnitedKingdom()
df_Merged.insert(0, "Holiday", [1 if str(val).split()[0] in UK_holidays else 0 for val in df_Merged.index.date])
df_Merged["Holiday"] = df_Merged["Holiday"].astype("category")

**Step 5:** Append temporal data to our merged dataframe.

In [ ]:
spring = range(79, 172)
summer = range(172, 266)
fall = range(266, 355)

def season(doy):
    if doy in spring:
        return "Spring"
    if doy in summer:
        return "Summer"
    if doy in fall:
        return "Fall"
    else:
        return "Winter"


df_Merged.insert(0, "Year", df_Merged.index.year)
df_Merged.insert(1, "Month", df_Merged.index.month)
df_Merged.insert(3, "Day", df_Merged.index.day)
df_Merged.insert(4, "Hour", df_Merged.index.hour)
df_Merged.insert(5, "Minute", df_Merged.index.minute)
df_Merged.insert(6, "Weekday", df_Merged.index.weekday)
# df_Merged.insert(8, "Season", df_Merged.index.dayofyear.map(season))

## Part 3: Visualize the data
**Step 1:** Reformat the index into a string so as to make sure that Matplotlib does not automatically interpolate missing values.

In [ ]:
df_REFIT_resampled_c = df_REFIT_resampled.copy()
df_REFIT_resampled_c.index = df_REFIT_resampled_c.index.strftime("%d-%m-%y %H:%M:%S")

In [ ]:
cols_REFIT_c = cols_REFIT.copy()
cols_REFIT_c.remove("Aggregate")
df_REFIT_resampled_c.drop(cols_REFIT_c, axis=1, inplace=True)

**Step 2:** Simple plot.

In [ ]:
fig, ax = plt.subplots()
df_REFIT_resampled_c["Aggregate"].plot(ax=ax)

ax.set_xlabel("")
ax.set_ylabel("Aggregate Power Consumption (Watts)")
ax.set_xlim(left=-5, right=len(df_REFIT_resampled_c) + 5)
ax.set_ylim(bottom=df_REFIT_resampled_c["Aggregate"].min() - 100)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.tight_layout()
plt.show()

**Step 3:** Stack plot.

In [ ]:
hours = []
for i in range(0, 24):
    if i < 10:
        hours.append(f"0{i}:00:00")
    if i >= 10:
        hours.append(f"{i}:00:00")

In [ ]:
df_Stacked = df_REFIT.iloc[:, 1:10].groupby(df_REFIT.index.hour).mean()

df_Stacked = df_Stacked.rename(
    index=str,
    columns={
        "Appliance1": "Fridge-Freezer",
        "Appliance2": "Unknown",
        "Appliance3": "Unknown",
        "Appliance4": "Computer Site",
        "Appliance5": "Microwave",
        "Appliance6": "Kettle",
        "Appliance7": "Toaster",
        "Appliance8": "Television",
        "Appliance9": "Unknown",
    },
)
df_Stacked.index = hours
df_Stacked.index = pd.to_datetime(df_Stacked.index, format="%H:%M:%S")
df_Stacked

In [ ]:
fig, ax = plt.subplots()

colors = ["b", "orange", "g", "r", "purple", "brown", "pink", "gray", "yellow"]

ax.stackplot(df_Stacked.index.values, np.transpose(df_Stacked.values), labels=df_Stacked.columns.values, colors=colors)
ax.legend(loc="upper left", fontsize=18)
ax.set_xlim(df_Stacked.index.min(), df_Stacked.index.max())
ax.xaxis.set_major_locator(md.HourLocator())
ax.xaxis.set_major_formatter(md.DateFormatter("%H:%M"))

plt.xlabel("Time")
plt.ylabel("Average Power Consumption (Watts)")
plt.xticks(df_Stacked.index.values)

fig.autofmt_xdate()

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 3)
i = 0
colors = ["b", "orange", "g", "r", "purple", "brown", "pink", "gray", "yellow"]

list = []
for x in range(0, 24, 6):
    list.append(df_Stacked.index[x])
list.append(df_Stacked.index[-1])

for nrow in range(0, 3):
    for ncol in range(0, 3):
        axs[nrow, ncol].plot(df_Stacked.index, df_Stacked.iloc[:, i], color=colors[i])
        axs[nrow, ncol].set_xlim(df_Stacked.index.min(), df_Stacked.index.max())
        axs[nrow, ncol].set_title(df_Stacked.columns[i], fontsize=18)
        axs[nrow, ncol].xaxis.set_major_locator(md.HourLocator())
        axs[nrow, ncol].xaxis.set_major_formatter(md.DateFormatter("%H:%M"))
        axs[nrow, ncol].set_xticks(list)
        i = i + 1

fig.autofmt_xdate()
fig.add_subplot(111, frameon=False)

plt.grid(False)
plt.tick_params(labelcolor="none", top=False, bottom=False, left=False, right=False)
plt.xlabel("Time", labelpad=25)
plt.ylabel("Average Power Consumption (Watts)", labelpad=15)
plt.tight_layout()
plt.show()

**Step 4.1:** Visualize the number of samples per day of the week over the entire data set.

In [ ]:
df_Merged_c = df_Merged.copy()
df_Merged_c.insert(0, "Weekday_name", df_Merged.index.day_name())

ax = sns.countplot(
    x=df_Merged_c["Weekday_name"],
    data=df_Merged_c,
    palette="icefire",
    order=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
)

plt.xlabel("Weekday")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

**Step 4.2:** Visualize the number of samples per month over the entire data set.

In [ ]:
df_Merged_c = df_Merged.copy()
df_Merged_c.insert(0, "Month_name", df_Merged.index.month_name())

ax = sns.countplot(
    x=df_Merged_c["Month_name"],
    data=df_Merged_c,
    palette="icefire",
    order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

**Step 5:** Test for stationarity.

In [ ]:
test_stationarity(df_REFIT_resampled_c, 0.05, "Aggregate", "Aggregate Power Consumption (Watts)")

## Part 4: Enumerate values of interest

**Step 1:** We iterate over each of the houses available in our data set and calculate the total number of missing days, total number of days that contain *incomplete* data, total number of values recorded as well as the number of values that contain *issues* and save the results to a `.csv` file.

In [ ]:
output = {
    "House number": [],
    "Date start": [],
    "Date end": [],
    "Total no. of days": [],
    "No. of days missing": [],
    "% of days missing": [],
    "No. of days incomplete": [],
    "% of days incomplete": [],
    "Total no. of values recorded": [],
    "No. of values missing data": [],
    "% of values missing data": [],
    "Number of values with issues": [],
    "% of values with issues": [],
    "Longest period of missing days": [],
}

for i in notebook.trange(1, 22):
    if i == 14:
        continue

    house_number = i
    house = f"CLEAN_House{house_number}.csv"
    file_destination = os.path.join(data_directory_REFIT, house)
    df = pd.read_csv(file_destination, index_col=0, parse_dates=True)

    # Define the starting date and ending date of our dataframe.
    date_start = df.index.min()
    date_end = df.index.max()

    df_grouped = df.groupby(df.index.date).mean()
    total_values = len(df)
    total_days = len(pd.date_range(df_grouped.index.min(), df_grouped.index.max()))
    
    # Set up a dataframe with which to calculate incomplete days.
    df_r = df.resample("15min").mean()
    df_r = df_r.dropna()

    # Count the number of missing days.
    days_missing = len(pd.date_range(df_grouped.index.min(), df_grouped.index.max()).difference(df_grouped.index))
    percentage_days_missing = np.round((days_missing / total_days) * 100, 2)

    # Count the number of days with incomplete data.
    mask = df_r.groupby(df_r.index.date).size()
    mask = mask[mask < 96].index.to_list()
    days_incomplete = len(mask)
    percentage_days_incomplete = np.round((days_incomplete / total_days) * 100, 2)

    # Count the number of missing values.
    values_missing = (df["Aggregate"] == 0).astype(int).sum()
    percentage_values_missing = np.round((values_missing / total_values) * 100, 2)

    # Count the number of values with 'issues'.
    values_issues = (df["Issues"] == 1).astype(int).sum()
    percentage_values_issues = np.round((values_issues / total_values) * 100, 2)

    # Calculate the longest period of missing days.
    df_grouped["Diff_Date"] = df_grouped.index.to_series().diff()
    df_grouped["Diff_Date"] = df_grouped["Diff_Date"] / np.timedelta64(1, "D")
    stretch = df_grouped["Diff_Date"].max().astype(int) - 1

    output["House number"].append(house_number)
    output["Date start"].append(date_start)
    output["Date end"].append(date_end)
    output["Total no. of days"].append(total_days)
    output["No. of days missing"].append(days_missing)
    output["% of days missing"].append(percentage_days_missing)
    output["Total no. of values recorded"].append(total_values)
    output["No. of days incomplete"].append(days_incomplete)
    output["% of days incomplete"].append(percentage_days_incomplete)
    output["No. of values missing data"].append(values_missing)
    output["% of values missing data"].append(percentage_values_missing)
    output["Number of values with issues"].append(values_issues)
    output["% of values with issues"].append(percentage_values_issues)
    output["Longest period of missing days"].append(stretch)
    
output = pd.DataFrame.from_dict(output).set_index("House number")
output.to_csv("Data Analysis - REFIT.csv")

## Part 5: Time series decomposition

**Step 1:** Define a period of 2-3 months (as an example).

In [ ]:
cols_REFIT_c = cols_REFIT.copy()
cols_REFIT_c.remove("Aggregate")

df_REFIT_resampled_c = df_REFIT_resampled.copy()
df_REFIT_resampled_c = df_REFIT_resampled_c.loc["2015-01-01":"2015-04-01"]
df_REFIT_resampled_c.drop(cols_REFIT_c, axis=1, inplace=True)
df_REFIT_resampled_c.index = df_REFIT_resampled_c.index.strftime("%d-%m-%y %H:%M:%S")

**Step 2:** Perform time series decomposition using LOESS.

In [ ]:
freq = (24 * 60) // 15
stl_decompose_result = STL(df_REFIT_resampled_c, period=freq).fit()

**Step 3:** Separate/plot the obtained results.

In [ ]:
observed = stl_decompose_result.observed
trend = stl_decompose_result.trend.to_frame()
seasonal = stl_decompose_result.seasonal.to_frame()
noise = stl_decompose_result.resid.to_frame()

In [ ]:
fig, axs = plt.subplots(4, sharex=True)

observed.plot(ax=axs[0], title="Observed")
trend.plot(ax=axs[1], title="Trend")
seasonal.plot(ax=axs[2], title="Seasonal")
noise.plot(ax=axs[3], title="Noise")

for ax in axs:
    ax.set_xlabel("")
    ax.set_xlim(0, right=len(observed))
    ax.get_legend().remove()
    ax.title.set_size(22)

fig.text(0.03, 0.6, "Aggregate Power Consumption (Watts)", fontsize="22", va="center", rotation="vertical")
plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.tight_layout()
plt.show()

## Part 6: Visualize outliers using box and whisker plots

**Step 1:** Box and whiskers plot for the aggregate as well as each of the IAM readings grouped by the months of the year.

In [ ]:
fig, axs = plt.subplots(len(cols_REFIT), 1, figsize=(16, 10 * (len(cols_REFIT))), sharex=True)

for col, ax in zip([*cols_REFIT], axs):
    sns.boxplot(
        data=df_REFIT,
        x=df_REFIT.index.month_name(),
        y=col,
        ax=ax,
        order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
    )
    ax.set_title(col)
    ax.xaxis.set_label_text("")

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.tight_layout()
plt.show()

**Step 2:** We remove outliers that are 3 standard deviations away from the mean and repeat **step 1**.

In [ ]:
df_out = df_REFIT[(np.abs(stats.zscore(df_REFIT["Aggregate"])) < 3)]
print(f"Number of outliers: {(len(df_REFIT) - len(df_out))}")

In [ ]:
fig, axs = plt.subplots(len(cols_REFIT), 1, figsize=(16, 10 * (len(cols_REFIT))), sharex=True)

for col, ax in zip([*cols_REFIT], axs):
    sns.boxplot(
        data=df_out,
        x=df_out.index.month_name(),
        y=col,
        ax=ax,
        order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
    )
    ax.xaxis.set_label_text("")

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.tight_layout()
plt.show()

**Step 2.1:** Purely for the aggregate power consumption.

In [ ]:
ax = sns.boxplot(
    data=df_out,
    x=df_out.index.month_name(),
    y=df_out["Aggregate"],
    order=["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"],
)

plt.setp(ax.get_xticklabels(), ha="right", rotation=60)
plt.xlabel("Month")
plt.ylabel("Aggregate Power Consumption (Watts)")
plt.tight_layout()
plt.show()

**Step 3:** Box and whiskers plot for the aggregate as well as each of the IAM readings over the entirety of the data set.

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=df_out, ax=ax)

plt.setp(ax.get_xticklabels(), ha="right", rotation=45)
plt.tight_layout()
plt.show()

## Part 7: Test for causality and correlation
**Step 1:** Perform the Augmented Dicky-Fuller test to determine whether our time series is stationary.

In [ ]:
for name, column in df_REFIT_resampled.iteritems():
    adfuller_test(column, name=column.name)
    print("\n")

**Step 2:** Check for autocorrelation.

In [ ]:
plot_acf(df_REFIT_resampled["Aggregate"], lags=50, zero=False)
plt.show()

**Step 3:** Check for partial autocorrelation.

In [ ]:
plot_pacf(df_REFIT_resampled["Aggregate"], lags=50, zero=False)
plt.show()

**Step 4:** Check for highly correlated features in the Solcast dataframe and drop them from our merged dataframe.

In [ ]:
highly_correlated = correlation(df_Solcast, 0.7)
df_Merged.drop(highly_correlated, axis=1, inplace=True)

**Step 5:** Estimate mutual information of our independent variables on our target variable.

In [ ]:
X, y = (df_Merged.loc[:, df_Merged.columns != "Aggregate"], df_Merged["Aggregate"])
mutual_info = mutual_info_regression(X, y)

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X.columns
mutual_info.sort_values(ascending=False)

In [ ]:
mutual_info.sort_values(ascending=False).plot.bar()
plt.tight_layout()
plt.show()

## Miscellaneous functions
**1)** Perform the Augmented Dickey–Fuller test on our data to determine whether it is stationary.

In [ ]:
def adfuller_test(series, signif=0.05, name=""):
    r = adfuller(series, autolag="AIC")
    output = {"test_statistic": round(r[0], 4), "pvalue": round(r[1], 4), "n_lags": round(r[2], 4), "n_obs": r[3]}
    p_value = output["pvalue"]

    def adjust(val, length=6):
        return str(val).ljust(length)

    print(f'      Augmented Dickey-Fuller Test on "{name}"', "\n   ", "-" * 47)
    print(f" Null Hypothesis: Data has unit root. Non-Stationary.")
    print(f" Significance Level    = {signif}")
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key, val in r[4].items():
        print(f" Critical value {adjust(key)} = {round(val, 3)}")
    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")

**2)** Perform the Cointegration test which helps to establish the presence of a statistically significant connection between two or more time series.

In [ ]:
def cointegration_test(df, alpha=0.05):
    out = coint_johansen(df, -1, 5)
    d = {"0.90": 0, "0.95": 1, "0.99": 2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1 - alpha)]]

    def adjust(val, length=6):
        return str(val).ljust(length)

    print("Name   ::  Test Stat > C(95%)    =>   Signif  \n", "--" * 20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ":: ", adjust(round(trace, 2), 9), ">", adjust(cvt, 8), " =>  ", trace > cvt)

**3)** Determine highly correlated features.

In [ ]:
def correlation(df, threshold, target_variable):
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                rowname = corr_matrix.index[j]
                cor1 = abs(df[colname].corr(target_variable))
                cor2 = abs(df[rowname].corr(target_variable))
                if  cor1 > cor2:
                    col_corr.add(corr_matrix.index[j])
                else:
                    col_corr.add(corr_matrix.columns[i])
    return col_corr

**4)** Perform the Granger Causality test to determine whether one time series is useful in forecasting another.

In [ ]:
def grangers_causation_matrix(data, variables, test="ssr_chi2test", maxlag=12, verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = (1 - [round(test_result[i + 1][0][test][1], 4) for i in range(maxlag)])
            if verbose:
                print(f"Y = {r}, X = {c}, P Values = {p_values}")
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + "_x" for var in variables]
    df.index = [var + "_y" for var in variables]
    return df

**5)** Test stationarity with plot.

In [ ]:
def test_stationarity(series, signif=0.05, name="", ylabel=""):
    def adjust(val, length=6):
        return str(val).ljust(length)

    rolmean = series.rolling(12).mean()
    rolstd = series.rolling(12).std()

    fig, ax = plt.subplots()

    series.plot(ax=ax, alpha=0.5)
    rolmean.plot(ax=ax, alpha=0.7)
    rolstd.plot(ax=ax, alpha=0.7)

    ax.set_xlabel("")
    ax.set_ylabel(ylabel)
    ax.set_xlim(left=0, right=len(series))
    plt.legend(loc="best")
    plt.title("Rolling Mean & Standard Deviation")
    plt.setp(ax.get_xticklabels(), ha="right", rotation=60)

    leg = plt.legend()
    leg.get_texts()[0].set_text(name)
    leg.get_texts()[1].set_text("Rolling Mean")
    leg.get_texts()[2].set_text("Rolling STD")

    plt.tight_layout()
    plt.show(block=False)

    adfuller_test(series, 0.05, name)

**6)** Reshape correlation matrix.

In [ ]:
def reshape_corr(df):
    df_corr = df.corr().stack().reset_index()
    df_corr.columns = ["Feature 1", "Feature 2", "Correlation"]
    mask_dups = (df_corr[["Feature 1", "Feature 2"]].apply(frozenset, axis=1).duplicated()) | (df_corr["Feature 1"] == df_corr["Feature 2"])
    df_corr = df_corr[~mask_dups]

    return df_corr

## Testing grounds

**1)** Using Bayesian Optimization and Ordinary Least Squares for feature selection/pruning. (Doesn't really work)

In [ ]:
import statsmodels.api as sm

from bayes_opt import BayesianOptimization
from itertools import combinations
from statsmodels.formula.api import ols

In [ ]:
X, y = (df_Merged.loc[:, df_Merged.columns != "Aggregate"], df_Merged["Aggregate"])

In [ ]:
b2 = []
for i in range(1, len(X.columns) - 15):
    b2.append(list(combinations(X.columns, i)))
b2 = [item for sublist in b2 for item in sublist]

In [ ]:
def example_function(i):
    max_r2 = 0
    max_list = None
    item = format("+".join(b2[int(i)]))
    model = ols(f"Aggregate ~ {item}", data=df_Merged).fit()
    r2 = model.rsquared_adj**.5
    if r2 > max_r2:
        max_r2 = r2
        max_list = b2[int(i)]
    return max_r2

def function_to_be_optimized(i):
    d = int(i)
    return example_function(d)

In [ ]:
pbounds = {'i': (1, len(b2) - 1)}

optimizer = BayesianOptimization(
    pbounds=pbounds,
    f=function_to_be_optimized,
    random_state=1,
)

optimizer.maximize(
    init_points=2,
    n_iter=200,
)

**2)** Dimensionality reduction and HDB

In [ ]:
import hdbscan
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
df_REFIT_resampled_c = df_REFIT_resampled.copy()
cols_REFIT_c = cols_REFIT.copy()
cols_REFIT_c.remove("Aggregate")
df_REFIT_resampled_c.drop(cols_REFIT_c, axis=1, inplace=True)

In [ ]:
df_REFIT_resampled_c.index = pd.MultiIndex.from_arrays([df_REFIT_resampled_c.index.date, df_REFIT_resampled_c.index.time], names=['Date','Time'])
df_REFIT_resampled_c = df_REFIT_resampled_c.unstack()
df_REFIT_resampled_c

In [ ]:
X = TSNE(n_components=2).fit_transform(df_REFIT_resampled_c)

In [ ]:
X.shape

In [ ]:
pca = PCA().fit(df_REFIT_resampled_c)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance');

In [ ]:
pca = PCA(n_components=2)
X = pca.fit_transform(df_REFIT_resampled_c)
X.shape

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=4)
clusterer = clusterer.fit(X)

In [ ]:
labels = clusterer.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)
print(n_clusters_, n_noise_, len(df_REFIT_resampled_c))

In [ ]:
df_REFIT_resampled_c['Labels'] = labels
for i in range(0, n_clusters_):
    globals()['df_c' + str(i)] = (df_REFIT_resampled_c.loc[df_REFIT_resampled_c['Labels'] == i])

In [ ]:
for q in range(len(df_c3)):
    row = df_c3.iloc[q,0:48].plot(alpha=0.7)
    row.plot()

In [ ]:
plt.scatter(X[:, 0],X[:, 1])
plt.show()

In [ ]:
plt.scatter(X[:, 0],X[:, 1])
plt.show()

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding
X = LocallyLinearEmbedding(n_components=2).fit_transform(df_REFIT_resampled_c)

In [ ]:
plt.scatter(X[:, 0],X[:, 1])
plt.show()

In [ ]:
from sklearn.decomposition import FactorAnalysis

In [ ]:
X = FactorAnalysis(n_components=2, random_state=0).fit_transform(df_REFIT_resampled_c)

In [ ]:
plt.scatter(X[:, 0],X[:, 1])
plt.show()